In [59]:
import numpy as np
import scipy
from scipy.stats import binned_statistic
from scipy.stats import binned_statistic_2d
import scipy.interpolate as interp
import scipy.stats as st
import scipy.signal as sig
from scipy.optimize import curve_fit


import h5py
import gc
import sys
import subprocess
import pickle
import os


import astropy as ap
from astropy.io import fits
from astropy import table as t
from astropy.table import Table
from astropy import wcs
from astropy.cosmology import WMAP7


import matplotlib.pyplot as plt
import matplotlib.patheffects as patheffects
from matplotlib import rcParams
from matplotlib.path import Path
import matplotlib.patches as patches
import matplotlib as mpl
from matplotlib import rc
from matplotlib import colors

rcParams["font.family"] = "DejaVu Sans"
rcParams["font.sans-serif"] = ["Computer Modern Sans"]
rcParams["text.usetex"] = True
rcParams["text.latex.preamble"] = r"\usepackage{cmbright}"
rcParams['xtick.major.pad']='10'
rcParams['ytick.major.pad']='10'
rcParams['font.size'] = 22
rcParams['axes.labelsize'] = 22
rcParams['axes.labelweight'] = 'bold'
rcParams['axes.titlesize'] = 17
rcParams['xtick.labelsize'] = 17
rcParams['ytick.labelsize'] = 17
rcParams['legend.fontsize'] = 15
rcParams['figure.titlesize'] = 1

In [22]:
os.chdir("/Volumes/500GB/nmd299/data/mpl7")

drpall = t.Table.read('drpall-v2_5_3.fits')
drpall.add_index('plateifu')

index = np.where(drpall['srvymode']=='MaNGA dither')[0]
plate_ifu = drpall['plateifu'][index]
# drp_logcube = fits.open('manga-7977-12701-LOGCUBE.fits')
# wave = drp_logcube['WAVE'].data

print(np.unique(drpall['srvymode']))

index = np.where(drpall['srvymode']=='MaNGA dither')[0]
plate_ifu = drpall['plateifu'][index]
index2 = np.where(drpall['srvymode']=='MaNGA 10min')[0]
plate_ifu2 = drpall['plateifu'][index2]

print(len(drpall))
print(len(index))
print(len(index2))
print(plate_ifu[0:1300])

print(plate_ifu[0:1500])
drpall2 = drpall[index]
print(len(drpall))
print(len(drpall2))
q = drpall2['drp3qual'][0:1300]
#np.unique(q)
print(len(plate_ifu[0:1300][q==66]))
print(len(plate_ifu[0:1300][q==64]))
print(len(plate_ifu[0:1300][q==2]))

print(len(plate_ifu[0:1300][q==1073742080]))
print(np.unique(q[q!=0]))



print(plate_ifu[0:1300][q==0])
print(plate_ifu[0:1300][q==64])
print(a)
drpall.info()
drpnos = plate_ifu[0:1300]

In [32]:
tings = os.listdir("/Volumes/500GB/nmd299/data/mpl7")[2:]
print(tings)
print(len(tings))
print(len(np.unique(tings)))
print(len(np.unique(drpnos)))
print(len(a))
print(a)
a = np.setdiff1d(drpnos,tings)

1194
1194
1300


In [8]:
np.unique(drpall['srvymode'])

MaNGA 10min
MaNGA dither


In [3]:
def dn4000_red(wave,spec):
        interp_spec = interp.interp1d(wave,spec)
        red_wav = np.linspace(4000,4100,100)
        d4000_r = np.sum(interp_spec(red_wav))
        return d4000_r

def dn4000_blue(wave,spec):
    interp_spec = interp.interp1d(wave,spec)
    blue_wav = np.linspace(3850,3950,100)
    d4000_b =  np.sum(interp_spec(blue_wav))
    return d4000_b

def dn_4000(wave,specs):
    dn_4000 = np.sum([dn4000_red(wave,specs[i]) for i in
        range(len(specs))]) / np.sum([dn4000_blue(wave,specs[i])
            for i in range(len(specs))])
    return dn_4000

In [4]:
#Hdelta
bandw_HdA = np.logical_and(wave > 4083.500, wave < 4122.250)
bandw_HdA_blueside = np.logical_and(wave > 4041.600, wave < 4079.750)
bandw_HdA_redside = np.logical_and(wave > 4128.500, wave < 4161.000)
blues = len(wave[bandw_HdA_blueside])
reds = len(wave[bandw_HdA_redside])

In [5]:
def tsum(xin, yin):
    tsum = np.sum(np.abs((xin[1:]-xin[:-1]))*(yin[1:]+yin[:-1])/2. )
    return tsum

In [6]:
def HdA(specs):
    spec_av_blueside = np.sum([np.sum(specs[i][bandw_HdA_blueside])
        for i in range(len(specs))])/(blues*len(specs))
    spec_av_redside = np.sum([np.sum(specs[i][bandw_HdA_redside])
        for i in range(len(specs))])/(reds*len(specs))

    a_spec = (spec_av_redside - spec_av_blueside)/(
            (4161.000+4128.500)/2.0 - (4079.750+4041.600)/2.0)
    b_spec = spec_av_blueside - a_spec * (4079.750+4041.600)/2
    spec_cont_HdA = wave[bandw_HdA] * a_spec + b_spec
    mean_dip = np.mean([specs[i][bandw_HdA] for i in range(len(specs))])
    HdA = tsum(wave[bandw_HdA], np.divide(spec_cont_HdA - mean_dip, spec_cont_HdA))
    return HdA

In [7]:
def get_specs(galaxy, NX, NY):
    x_ax = np.arange(0, NX) + 0.5 - (NX/2.0)
    y = np.outer((np.arange(0, NX) + 0.5) - (NX/2.0), np.ones(NX))
    x = np.transpose(y)
    grid = np.sqrt((x*x+y*y))
    #unravel the cube
    spectra = []
    for i in range(NX):
        for j in range(NY):
            thing = galaxy[:,i,j]
            spectra.append(thing)

    spectra = np.array(spectra)
    #3 arcsec aperture
    radii = np.ravel(grid)
    index = np.where(radii<=3.0)[0]
    spaxels_within = spectra[index]
    return(spaxels_within)


In [8]:
fits_files = ['manga-'+ str(plate_ifu[i]) + '-LOGCUBE.fits' for i in range(470)]

In [9]:
n = []
dn4 = []
hda = []

# drp_logcube = fits.open(fits_files[i])
# wave = drp_logcube['WAVE'].data

for i in range(321,325):
    drp_logcube = fits.open(fits_files[i])
    wave = drp_logcube['WAVE'].data
    print(i)
    #print(wavez)
#     NL, NY, NX = drp_logcube['FLUX'].data.shape
#     n.append([NX,NY])
#     flux = drp_logcube['FLUX'].data
#     specs = get_specs(flux, NX, NY)
#     hdelta = HdA(specs)
#     dn4000 = dn_4000(wave,specs)
#     dn4.append(dn4000)
#     hda.append(hdelta)

#print(dn4, hda)
dn4 = np.array(dn4)
hda = np.array(hda)
stuff = np.column_stack((dn4,hda))

321
322
323
324


In [ ]:
plt.scatter(dn4,hda)
plt.show()

In [21]:
print(plate_ifu[0:250], min(plate_ifu[0:250]), max(plate_ifu[0:250]))
woo = [plate_ifu[i] for i in range(318,322)]
print(fits_files[320], len(woo))
drp_logcube = fits.open(fits_files[320])
wave = drp_logcube['WAVE'].data

4